In [ ]:

!unzip -q /content/drive/MyDrive/235701_모션키포인트.zip

In [ ]:
import os
from os.path import exists, join, basename, splitext
import sys
import time
import matplotlib
import matplotlib.pylab as plt

#project 디렉토리생성 및 pose hrnet git clone
!mkdir -p drive/MyDrive/deep-high-resolution-net
!git clone https://github.com/leoxiaobin/deep-high-resolution-net.pytorch.git drive/MyDrive/deep-high-resolution-net
# project_name = '/content/drive/MyDrive/deep-high-resolution-net'
project_name = '/content/drive/MyDrive/deep-high-resolution-net'


fatal: destination path 'drive/MyDrive/deep-high-resolution-net' already exists and is not an empty directory.


In [ ]:
pre_trained_hrnet_path = os.path.join(project_name,'pre_train_model/hrnet_w48-8ef0771d.pth')
pre_trained_hrnet_pose_path = os.path.join(project_name,'pre_train_model/pose_hrnet_w48_384x288.pth')

In [ ]:
!pip install -q EasyDict==1.7 Cython pyyaml json_tricks scikit-image yacs>=0.1.5 ipdb
!cd {project_name}/lib && make

sys.path.append(project_name)
sys.path.append(join(project_name, 'tools'))
sys.path.append(join(project_name, 'lib'))
!pip install tensorboardx

cd nms; python setup_linux.py build_ext --inplace; rm -rf build; cd ../../
running build_ext
cythoning cpu_nms.pyx to cpu_nms.c
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/MyDrive/deep-high-resolution-net/lib/nms/cpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
cythoning gpu_nms.pyx to gpu_nms.cpp
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/MyDrive/deep-high-resolution-net/lib/nms/gpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'cpu_nms' extension
creating build
creating build/temp.linux-x86_64-cpython-310
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-stron

In [ ]:
import cv2
import numpy as np
from lib.core.inference import get_final_preds
import torch
import config
import models
from config import cfg, update_config
from types import SimpleNamespace

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from torch import nn, Tensor
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

/content/drive/MyDrive/deep-high-resolution-net/lib/models/pose_hrnet.py:487: SyntaxWarning: "is" with a literal. Did you mean "=="?
  or self.pretrained_layers[0] is '*':


In [ ]:
dataDir = '/content/1. open/'
train = pd.read_csv(dataDir + 'train_df.csv')
submission = pd.read_csv(dataDir + 'sample_submission.csv')

train.head(2)

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,001-1-1-01-Z17_A-0000001.jpg,1046.389631,344.757881,1041.655294,329.820225,1059.429507,334.48423,1020.117796,338.890539,1048.000000,...,1067.00000,335.0,1019.484230,455.000000,1026.515770,514.054730,998.578836,826.718013,1063.204067,838.827465
1,001-1-1-01-Z17_A-0000003.jpg,1069.850679,340.711494,1058.608552,324.593690,1075.242111,325.59369,1041.422997,331.694815,1065.593682,...,1081.18738,323.0,1046.953248,454.062706,1058.766231,508.797029,1002.265676,699.062706,1066.376234,841.499445


In [ ]:
def model_load(pre_trained_hrnet, pre_trained_hrnet_pose):
    yaml_file_name = os.path.join(project_name,'experiments/coco/hrnet/w48_384x288_adam_lr1e-3.yaml')

    update_config(cfg,
                SimpleNamespace(cfg = join(project_name, yaml_file_name),
                                opts = '',
                                modelDir='', logDir='', dataDir='', prevModelDir=''))

    pretrained_model_path = pre_trained_hrnet
    cfg['MODEL']['PRETRAINED'] = pretrained_model_path
    model_coco = eval('models.'+cfg.MODEL.NAME+'.get_pose_net')(
            cfg, is_train=True
        )

    pretrained_posemodel_path = pre_trained_hrnet_pose

    state_dict = torch.load(pretrained_posemodel_path)
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k # remove module.
        #print(name,'\t')
        new_state_dict[name] = v
    model_coco.load_state_dict(new_state_dict)

    cfg['MODEL']['NUM_JOINTS'] = 24
    cfg['DATASET']['NUM_JOINTS_HALF_BODY'] = 12

    model = eval('models.'+cfg.MODEL.NAME+'.get_pose_net')(
            cfg, is_train=True
        )

    final_layer = nn.Conv2d(
                in_channels=48,
                out_channels=24,
                kernel_size=1,
                stride=1,
                padding=0
            )

    model_coco.final_layer = model.final_layer
    model = model_coco

    ct = 0
    #model layer freeze
    for child in model.children():
        ct += 1
        #print(ct)
        if ct <9: #13 final layer
            for param in child.parameters():
                param.requires_grad = False
                #print(child)
        else:
            #print('False : ',child)
            pass

    model = model.cuda()

    return model

In [ ]:
from dataset.JointsDataset import JointsDataset
import logging
logger = logging.getLogger(__name__)
error_list=[317, 869, 873, 877, 911, 1559, 1560, 1562, 1566, 1575, 1577, 1578, 1582, 1606, 1607, 1622, 1623, 1624, 1625, 1629, 3968, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168, 4169, 4170, 4171, 4172, 4173, 4174, 4175, 4176, 4177, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4186, 4187, 4188, 4189, 4190, 4191, 4192, 4193, 4194]


class DaconDataset(JointsDataset):
    '''
    "keypoints": {
        0: "nose",              nose_x              1.
        1: "left_eye",          left_eye_x          1.
        2: "right_eye",         right_eye_x         1.
        3: "left_ear",          left_ear_x          1.
        4: "right_ear",         right_ear_x         1.
        5: "left_shoulder",     left_shoulder_x     1.
        6: "right_shoulder",    right_shoulder_x    1.
        7: "left_elbow",        left_elbow_x        1.2
        8: "right_elbow",       right_elbow_x       1.2
        9: "left_wrist",        left_wrist_x        1.5
        10: "right_wrist",      right_wrist_x       1.5
        11: "left_hip",         left_hip_x          1.
        12: "right_hip",        right_hip_x         1.
        13: "left_knee",        left_knee_x         1.2
        14: "right_knee",       right_knee_x        1.2
        15: "left_ankle",       left_ankle_x        1.5
        16: "right_ankle"       right_ankle_x       1.5
        17:                     neck_x              1.
        18:                     left_palm_x         1.5
        19:                     right_palm_x        1.5
        20:                     spine2(back)_x      1.
        21:                     spine1(waist)_x     1.
        22:                     left_instep_x       1.5
        23:                     right_instep_x      1.5

    },
	"skeleton": [
        [16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13], [6,7],[6,8],
        [7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7]]
    '''
    def __init__(self, cfg, root, image_set, k_fold, is_train, transform=None):
        super().__init__(cfg, root, image_set, is_train, transform)
        self.nms_thre = cfg.TEST.NMS_THRE
        self.image_thre = cfg.TEST.IMAGE_THRE
        self.soft_nms = cfg.TEST.SOFT_NMS
        self.oks_thre = cfg.TEST.OKS_THRE
        self.in_vis_thre = cfg.TEST.IN_VIS_THRE
        self.bbox_file = cfg.TEST.COCO_BBOX_FILE
        self.use_gt_bbox = cfg.TEST.USE_GT_BBOX
        self.image_width = cfg.MODEL.IMAGE_SIZE[0]
        self.image_height = cfg.MODEL.IMAGE_SIZE[1]
        self.aspect_ratio = self.image_width * 1.0 / self.image_height
        self.pixel_std = 200

        label_path = root + '/train_df.csv'
        self.image_dir = root + 'train_imgs'
        self.df = pd.read_csv(label_path)


        len_df = len(self.df)
        newdf = pd.DataFrame(columns=self.df.columns)

        error_index = 0
        index_cnt = 0
        sample_num = 5
        start_idx = k_fold
        print(k_fold)
        for i in range(start_idx,len_df):
            if error_list[error_index] == i:
                error_index += 1
                continue

            if is_train:
                if index_cnt % sample_num != 0:
                    newdf = newdf.append(self.df.iloc[i,:], ignore_index=True)
                #newdf = newdf.append(self.df.iloc[i,:], ignore_index=True)
            else:
                if index_cnt % sample_num == 0:
                    newdf = newdf.append(self.df.iloc[i,:], ignore_index=True)

            index_cnt += 1

        self.df = newdf
        print(self.df.head(2))

        self.width = 1920
        self.height = 1080

        # load image file names
        self.image_set_index = self._load_image_set_index()
        self.num_images = len(self.image_set_index)
        logger.info('=> num_images: {}'.format(self.num_images))
        print('=> num_images: {}'.format(self.num_images))

        self.num_joints = 24
        self.flip_pairs = [[1, 2], [3, 4], [5, 6], [7, 8],
                           [9, 10], [11, 12], [13, 14], [15, 16], [18, 19], [22, 23]]
        self.parent_ids = None
        self.upper_body_ids = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 17, 18, 19, 20)
        self.lower_body_ids = (11, 12, 13, 14, 15, 16, 21, 22, 23)

        self.joints_weight = np.array(
            [
                1., 1., 1., 1., 1., 1., 1., 1.2, 1.2,
                1.5, 1.5, 1., 1., 1.2, 1.2, 1.5, 1.5,
                1.2, 1.5, 1.5, 1.2, 1.2, 1.5, 1.5,
            ],
            dtype=np.float32
        ).reshape((self.num_joints, 1))

        self.db = self._get_db()

        if is_train and cfg.DATASET.SELECT_DATA:
            self.db = self.select_data(self.db)

        logger.info('=> load {} samples'.format(len(self.db)))
        print('=> load {} samples'.format(len(self.db)))

    def _load_image_set_index(self):
        """ image id: int """
        len_imgs = self.df.shape[0]
        image_ids = list(range(len_imgs))
        return image_ids

    def _get_db(self):
        if self.is_train or self.use_gt_bbox:
            # use ground truth bbox
            gt_db = self._load_dacon_keypoint_annotations()
        else:
            #pass
            # use bbox from detection
            gt_db = self._load_dacon_keypoint_annotations()
        return gt_db

    def _load_dacon_keypoint_annotations(self):
        """ ground truth bbox and keypoints """
        gt_db = []
        for index in self.image_set_index:
            gt_db.extend(self._load_dacon_keypoint_annotation_kernal(index))
        return gt_db

    def _load_dacon_keypoint_annotation_kernal(self, index):
        keypoints = self.df.iloc[index, 1:].values.reshape(-1, 2).astype(np.int64)

        x1, y1 = min(keypoints[:, 0]), min(keypoints[:, 1])
        x2, y2 = max(keypoints[:, 0]), max(keypoints[:, 1])

        w = x2 - x1
        h = y2 - y1
        boxes = [x1, y1, w, h]

        rec = []

        joints_3d = np.zeros((self.num_joints, 3), dtype=np.float)
        joints_3d_vis = np.zeros((self.num_joints, 3), dtype=np.float)

        for ipt in range(self.num_joints):
            joints_3d[ipt, 0] = keypoints[ipt][0]
            joints_3d[ipt, 1] = keypoints[ipt][1]
            joints_3d[ipt, 2] = 0
            t_vis = 1
            if(keypoints[ipt][0] <= 0 and keypoints[ipt][1] <= 0):
                t_vis = 0

            joints_3d_vis[ipt, 0] = t_vis
            joints_3d_vis[ipt, 1] = t_vis
            joints_3d_vis[ipt, 2] = 0

        center, scale = self._box2cs(boxes)
        rec.append({
            'image': self.image_path_from_index(index),
            'center': center,
            'scale': scale,
            'joints_3d': joints_3d,
            'joints_3d_vis': joints_3d_vis,
            'filename': '',
            'imgnum': 0,
        })

        return rec

    def _box2cs(self, box):
        x, y, w, h = box[:4]
        return self._xywh2cs(x, y, w, h)

    def _xywh2cs(self, x, y, w, h):
        center = np.zeros((2), dtype=np.float32)
        center[0] = x + w * 0.5
        center[1] = y + h * 0.5

        if w > self.aspect_ratio * h:
            h = w * 1.0 / self.aspect_ratio
        elif w < self.aspect_ratio * h:
            w = h * self.aspect_ratio
        scale = np.array(
            [w * 1.0 / self.pixel_std, h * 1.0 / self.pixel_std],
            dtype=np.float32)
        if center[0] != -1:
            scale = scale * 1.25

        return center, scale

    def image_path_from_index(self, index):
        image_id = self.df.iloc[index, 0]
        image_path = os.path.join(self.image_dir, image_id)

        return image_path


In [ ]:
import torch.nn as nn


class JointsMSELoss(nn.Module):
    def __init__(self, use_target_weight):
        super(JointsMSELoss, self).__init__()
        self.criterion = nn.MSELoss(reduction='mean')
        self.use_target_weight = use_target_weight

    def forward(self, output, target, target_weight):
        batch_size = output.size(0)
        num_joints = output.size(1)
        heatmaps_pred = output.reshape((batch_size, num_joints, -1)).split(1, 1)
        heatmaps_gt = target.reshape((batch_size, num_joints, -1)).split(1, 1)
        loss = 0

        for idx in range(num_joints):
            heatmap_pred = heatmaps_pred[idx].squeeze()
            heatmap_gt = heatmaps_gt[idx].squeeze()
            if self.use_target_weight:
                loss += 0.5 * self.criterion(
                    heatmap_pred.mul(target_weight[:, idx]),
                    heatmap_gt.mul(target_weight[:, idx])
                )
            else:
                loss += 0.5 * self.criterion(heatmap_pred, heatmap_gt)

        return loss / num_joints

In [ ]:
from core.evaluate import accuracy
import logging
logger = logging.getLogger(__name__)
from utils.vis import save_debug_images

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count if self.count != 0 else 0


In [ ]:
def train(config, train_loader, model, criterion, optimizer, epoch,
          output_dir, writer_dict):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target, target_weight, meta) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input = input.cuda()
        # compute output
        outputs = model(input)

        target = target.cuda(non_blocking=True)
        target_weight = target_weight.cuda(non_blocking=True)

        if isinstance(outputs, list):
            loss = criterion(outputs[0], target, target_weight)
            for output in outputs[1:]:
                loss += criterion(output, target, target_weight)
        else:
            output = outputs
            loss = criterion(output, target, target_weight)

        # loss = criterion(output, target, target_weight)

        # compute gradient and do update step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        losses.update(loss.item(), input.size(0))

        _, avg_acc, cnt, pred = accuracy(output.detach().cpu().numpy(),
                                         target.detach().cpu().numpy())
        acc.update(avg_acc, cnt)

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        PRINT_FREQ = 10
        if i % PRINT_FREQ == 0:
            msg = 'Epoch: [{0}][{1}/{2}]\t' \
                  'Time {batch_time.val:.3f}s ({batch_time.avg:.3f}s)\t' \
                  'Speed {speed:.1f} samples/s\t' \
                  'Data {data_time.val:.3f}s ({data_time.avg:.3f}s)\t' \
                  'Loss {loss.val:.5f} ({loss.avg:.5f})\t' \
                  'Accuracy {acc.val:.3f} ({acc.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      speed=input.size(0)/batch_time.val,
                      data_time=data_time, loss=losses, acc=acc)
            logger.info(msg)
            print(msg)

            writer = writer_dict['writer']
            global_steps = writer_dict['train_global_steps']
            writer.add_scalar('train_loss', losses.val, global_steps)
            writer.add_scalar('train_acc', acc.val, global_steps)
            writer_dict['train_global_steps'] = global_steps + 1

            prefix = '{}_{}'.format(os.path.join(output_dir, 'train'), i)
            save_debug_images(config, input, meta, target, pred*4, output,
                              prefix)
    return acc.avg

def val(config, val_loader, model, criterion, output_dir,
            writer_dict=None):
    batch_time = AverageMeter()
    losses = AverageMeter()
    acc =  AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target, target_weight, meta) in enumerate(val_loader):
            # compute output
            input = input.cuda()
            outputs = model(input)
            if isinstance(outputs, list):
                output = outputs[-1]
            else:
                output = outputs

            if config.TEST.FLIP_TEST:
                input_flipped = input.flip(3)
                outputs_flipped = model(input_flipped)

                if isinstance(outputs_flipped, list):
                    output_flipped = outputs_flipped[-1]
                else:
                    output_flipped = outputs_flipped

                output_flipped = flip_back(output_flipped.cpu().numpy(),
                                           val_dataset.flip_pairs)
                output_flipped = torch.from_numpy(output_flipped.copy()).cuda()


                # feature is not aligned, shift flipped heatmap for higher accuracy
                if config.TEST.SHIFT_HEATMAP:
                    output_flipped[:, :, :, 1:] = \
                        output_flipped.clone()[:, :, :, 0:-1]

                output = (output + output_flipped) * 0.5

            target = target.cuda(non_blocking=True)
            target_weight = target_weight.cuda(non_blocking=True)

            loss = criterion(output, target, target_weight)

            num_images = input.size(0)
            # measure accuracy and record loss
            losses.update(loss.item(), num_images)
            _, avg_acc, cnt, pred = accuracy(output.cpu().numpy(),
                                             target.cpu().numpy())

            acc.update(avg_acc, cnt)

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            PRINT_FREQ = 20
            if i % PRINT_FREQ == 0:
                msg = 'Test: [{0}/{1}]\t' \
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t' \
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t' \
                      'Accuracy {acc.val:.3f} ({acc.avg:.3f})'.format(
                          i, len(val_loader), batch_time=batch_time,
                          loss=losses, acc=acc)
                logger.info(msg)
                print(msg)

                prefix = '{}_{}'.format(
                    os.path.join(output_dir, 'val'), i
                )
                save_debug_images(config, input, meta, target, pred*4, output,
                                  prefix)
    return acc.avg, losses.avg

def flip_back(output_flipped, matched_parts):
    '''
    ouput_flipped: numpy.ndarray(batch_size, num_joints, height, width)
    '''
    assert output_flipped.ndim == 4,\
        'output_flipped should be [batch_size, num_joints, height, width]'

    output_flipped = output_flipped[:, :, :, ::-1]

    for pair in matched_parts:
        tmp = output_flipped[:, pair[0], :, :].copy()
        output_flipped[:, pair[0], :, :] = output_flipped[:, pair[1], :, :]
        output_flipped[:, pair[1], :, :] = tmp

    return output_flipped

In [ ]:
#output folder 설정
final_output_dir = join(project_name,'outDir')
eval_out_dir = join(final_output_dir,'eval')
tb_log_dir = join(final_output_dir,'logs')
from tensorboardX import SummaryWriter


writer_dict = {
        'writer': SummaryWriter(log_dir=tb_log_dir),
        'train_global_steps': 0,
        'valid_global_steps': 0,
    }

In [ ]:
def save_checkpoint(states, is_best, output_dir, fold,
                    filename='checkpoint.pth'):
    torch.save(states, os.path.join(output_dir, filename))
    if is_best and 'state_dict' in states:
        torch.save(states['best_state_dict'],
                   os.path.join(output_dir, f'model_best_fold{fold}.pth'))

In [ ]:

last_epoch = -1
begin_epoch=0
end_epoch = 35
k_fold = 5
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )

for fold_index in range(0,k_fold):

    train_dataset = DaconDataset(
        cfg, dataDir, cfg.DATASET.TRAIN_SET, fold_index, True,
        transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=36,
        shuffle=True,
        num_workers=2,
        pin_memory=cfg.PIN_MEMORY
    )

    val_dataset = DaconDataset(
        cfg, dataDir, cfg.DATASET.TRAIN_SET, fold_index, False,
        transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=36,
        shuffle=False,
        num_workers=2,
        pin_memory=cfg.PIN_MEMORY
    )

    model = model_load(pre_trained_hrnet_path, pre_trained_hrnet_pose_path)

    optimizer = optim.Adam(model.parameters(),lr=0.001)
    criterion = JointsMSELoss(
            use_target_weight=True
        ).cuda()


    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, [20,30], 0.1,
            last_epoch=last_epoch
        )

    best_avg = 0.
    for epoch in range(begin_epoch, end_epoch):
        print('epoch: ',epoch)
        best_model = False
        # train for one epoch
        avg = train(cfg, train_loader, model, criterion, optimizer, epoch,
                final_output_dir,  writer_dict)
        lr_scheduler.step()

        val_acc, val_loss = val(cfg, val_loader, model, criterion, final_output_dir)

        #best model save
        if best_avg < val_acc:
            best_model = True
            best_avg = val_acc
            print('best_avg', best_avg)

        #print('avg', avg)

        logger.info('=> saving checkpoint to {}'.format(final_output_dir))
        print('=> saving checkpoint to {}'.format(final_output_dir))
        save_checkpoint({
            'epoch': epoch + 1,
            'model': cfg.MODEL.NAME,
            'state_dict': model.state_dict(),
            'best_state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, best_model, final_output_dir, fold_index)

0


<ipython-input-12-cb0d3812e9b9>:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newdf = newdf.append(self.df.iloc[i,:], ignore_index=True)


                          image       nose_x      nose_y   left_eye_x  \
0  001-1-1-01-Z17_A-0000003.jpg  1069.850679  340.711494  1058.608552   
1  001-1-1-01-Z17_A-0000005.jpg  1084.475902  337.000008  1078.717997   

   left_eye_y  right_eye_x  right_eye_y   left_ear_x  left_ear_y  right_ear_x  \
0  324.593690  1075.242111   325.593690  1041.422997  331.694815  1065.593682   
1  323.757889  1095.648412   325.242119  1061.039884  329.351571  1086.461032   

   ...  right_palm_x  right_palm_y  spine2(back)_x  spine2(back)_y  \
0  ...    1081.18738         323.0     1046.953248      454.062706   
1  ...    1101.00000         334.0     1044.538960      442.054730   

   spine1(waist)_x  spine1(waist)_y  left_instep_x  left_instep_y  \
0      1058.766231       508.797029    1002.265676     699.062706   
1      1052.844144       495.890539     989.437847     808.757889   

   right_instep_x  right_instep_y  
0     1066.376234      841.499445  
1     1066.071417      841.749554  

[2 rows 

<ipython-input-12-cb0d3812e9b9>:153: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  joints_3d = np.zeros((self.num_joints, 3), dtype=np.float)
<ipython-input-12-cb0d3812e9b9>:154: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  joints_3d_vis = np.zeros((self.num_joints, 3), dtype=np.float)


=> load 3258 samples
0


...

<ipython-input-12-cb0d3812e9b9>:153: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  joints_3d = np.zeros((self.num_joints, 3), dtype=np.float)
<ipython-input-12-cb0d3812e9b9>:154: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  joints_3d_vis = np.zeros((self.num_joints, 3), dtype=np.float)


=> load 814 samples
epoch:  0
Epoch: [0][0/91]	Time 1.944s (1.944s)	Speed 18.5 samples/s	Data 1.512s (1.512s)	Loss 0.00178 (0.00178)	Accuracy 0.091 (0.091)
Epoch: [0][10/91]	Time 0.426s (0.798s)	Speed 84.6 samples/s	Data 0.084s (0.443s)	Loss 0.00131 (0.00154)	Accuracy 0.468 (0.324)
Epoch: [0][20/91]	Time 0.477s (0.742s)	Speed 75.5 samples/s	Data 0.135s (0.391s)	Loss 0.00096 (0.00133)	Accuracy 0.646 (0.457)
Epoch: [0][30/91]	Time 0.487s (0.720s)	Speed 73.9 samples/s	Data 0.146s (0.372s)	Loss 0.00067 (0.00116)	Accuracy 0.750 (0.550)
Epoch: [0][40/91]	Time 0.401s (0.707s)	Speed 89.8 samples/s	Data 0.059s (0.360s)	Loss 0.00057 (0.00102)	Accuracy 0.796 (0.610)
Epoch: [0][50/91]	Time 0.399s (0.698s)	Speed 90.3 samples/s	Data 0.056s (0.351s)	Loss 0.00053 (0.00093)	Accuracy 0.806 (0.648)
Epoch: [0][60/91]	Time 0.342s (0.693s)	Speed 105.3 samples/s	Data 0.000s (0.346s)	Loss 0.00044 (0.00086)	Accuracy 0.867 (0.678)
Epoch: [0][70/91]	Time 0.514s (0.693s)	Speed 70.0 samples/s	Data 0.172s (0.347s)	

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboardX/event_file_writer.py", line 219, in run
    self._record_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboardX/event_file_writer.py", line 69, in flush
    self._py_recordio_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboardX/record_writer.py", line 193, in flush
    self._writer.flush()
OSError: [Errno 107] Transport endpoint is not connected


=> saving checkpoint to /content/drive/MyDrive/deep-high-resolution-net/outDir
epoch:  32
Epoch: [32][0/91]	Time 1.869s (1.869s)	Speed 19.3 samples/s	Data 1.464s (1.464s)	Loss 0.00011 (0.00011)	Accuracy 0.995 (0.995)
Epoch: [32][10/91]	Time 0.342s (0.835s)	Speed 105.4 samples/s	Data 0.000s (0.485s)	Loss 0.00015 (0.00013)	Accuracy 0.967 (0.978)
Epoch: [32][20/91]	Time 0.512s (0.761s)	Speed 70.3 samples/s	Data 0.171s (0.414s)	Loss 0.00014 (0.00013)	Accuracy 0.970 (0.978)
Epoch: [32][30/91]	Time 0.532s (0.739s)	Speed 67.7 samples/s	Data 0.189s (0.392s)	Loss 0.00013 (0.00013)	Accuracy 0.975 (0.978)
Epoch: [32][40/91]	Time 0.427s (0.721s)	Speed 84.2 samples/s	Data 0.086s (0.375s)	Loss 0.00014 (0.00014)	Accuracy 0.980 (0.977)
Epoch: [32][50/91]	Time 0.342s (0.708s)	Speed 105.3 samples/s	Data 0.000s (0.363s)	Loss 0.00012 (0.00013)	Accuracy 0.979 (0.976)


KeyboardInterrupt: ignored

In [ ]:
def detect_person(human_model, image):

    img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

    img = transforms.functional.to_tensor(img)
    img = img.cuda()

    predictions = human_model(img.unsqueeze(0))

    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].detach().cpu().numpy()
    boxes = predictions[0]['boxes'].detach().cpu().int().numpy()

    personIdxs = np.where(labels == 1)[0]

    if personIdxs.size > 0:
        person_scores = scores[personIdxs]
        person_boxes = boxes[personIdxs]
        personIdx = np.argmax(person_scores)
        bboxs = person_boxes[personIdx].tolist()

        return bboxs

    else:
        return None


In [ ]:
from utils.transforms import get_affine_transform
def PreProcess(image, bbox, cfg):

    data_numpy = image

    inputs = []
    centers = []
    scales = []

    x1,y1,x2,y2 = bbox
    box = [x1, y1, x2-x1, y2-y1]

    # box from image  --> return center, scale
    c, s = _box2cs(box, data_numpy.shape[0], data_numpy.shape[1])
    centers.append(c)
    scales.append(s)
    r = 0

    trans = get_affine_transform(c, s, r, cfg.MODEL.IMAGE_SIZE)

    input = cv2.warpAffine(
        data_numpy,
        trans,
        (int(cfg.MODEL.IMAGE_SIZE[0]), int(cfg.MODEL.IMAGE_SIZE[1])),
        flags=cv2.INTER_LINEAR)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]),
        ])
    input = transform(input).unsqueeze(0)
    inputs.append(input)


    inputs = torch.cat(inputs)
    return inputs, data_numpy, centers, scales

def _box2cs(box, image_width, image_height):
    x, y, w, h = box[:4]
    return _xywh2cs(x, y, w, h, image_width, image_height)

def _xywh2cs(x, y, w, h, image_width, image_height):
    center = np.zeros((2), dtype=np.float32)
    center[0] = x + w * 0.5
    center[1] = y + h * 0.5

    aspect_ratio = image_width * 1.0 / image_height
    pixel_std = 200

    if w > aspect_ratio * h:
        h = w * 1.0 / aspect_ratio
    elif w < aspect_ratio * h:
        w = h * aspect_ratio
    scale = np.array(
        [w * 1.0 / pixel_std, h * 1.0 / pixel_std],
        dtype=np.float32)
    if center[0] != -1:
        scale = scale * 1.25

    return center, scale

In [ ]:
from lib.core.inference import get_final_preds
!pip install tqdm
import operator
def inference(model, dataDir, final_output_dir, fold):
    submission = pd.read_csv(dataDir + 'sample_submission.csv')
    testDir = os.path.join(dataDir,'test_imgs')

    all_predictions = []
    for i in range(0,fold):

        model_name = f'model_best_fold{i}.pth'
        model_path = os.path.join(final_output_dir, model_name)

        state_dict = torch.load(model_path)
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k # remove module.
            #  print(name,'\t')
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)
        model.eval().cuda()

        #person detector model : fasterrcnn_resnet50_fpn
        import torchvision
        human_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        human_model.eval().cuda()

        len_imgs = submission.shape[0]
        files = []
        predictions = []

        for index in tqdm(range(0, len_imgs)):
            image_id = submission.iloc[index, 0]
            imgName = os.path.join(testDir, image_id)
            files.append(image_id)

            image = cv2.imread(imgName, cv2.IMREAD_COLOR)

            person_box = detect_person(human_model, image)
            #person detection 실패시 아래와 같이 default box를 생성하도록 하였습니다.
            if person_box is None:
                person_box = [530,260,1280,935]

            inputs, origin_img, center, scale = PreProcess(image,person_box, cfg)

            with torch.no_grad():
                output = model(inputs.cuda())
                preds, maxvals = get_final_preds(
                    cfg, output.clone().cpu().numpy(), np.asarray(center), np.asarray(scale))

            predictions.append(preds.reshape(-1))

        if i == 0:
            all_predictions = predictions
            print(all_predictions[0])
        else:
            for i, prediction in enumerate(predictions):
                all_predictions[i] += prediction
            print(all_predictions[0])

    for i, all_prediction in enumerate( all_predictions):
        all_predictions[i] = all_prediction/(fold)

    print(all_predictions[0])

    df = pd.DataFrame(columns=submission.columns)
    df['image'] = files
    df.iloc[:, 1:] = all_predictions
    df.head()
    ret = df.to_csv(os.path.join(final_output_dir,f'submission.csv'), index=False)


In [ ]:
inference(model, dataDir, final_output_dir, k_fold)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 72.6MB/s]
100%|██████████| 1600/1600 [02:59<00:00,  8.93it/s]


[1105.2283   528.97485 1095.4193   548.5929  1124.8463   548.5929
 1075.8013   592.7335  1149.3689   578.01996  967.9019   568.21094
 1110.1328   543.68835  938.47485  445.59808 1051.2787   430.88455
  923.7613   322.98523  992.4245   313.1762   820.7665   524.0703
  894.3342   504.45227  707.96265  524.0703   855.0981   489.7387
  707.96265  685.91925  840.3845   651.58765 1070.8967   582.9245
  943.37933  298.46268  987.51996  303.3672   987.51996  548.5929
  923.7613   524.0703   683.4401   700.6328   825.671    661.3967 ]


100%|██████████| 1600/1600 [02:59<00:00,  8.92it/s]


[2195.7432  1043.2361  2185.934   1097.1858  2239.8838  1082.4722
 2151.6025  1190.3716  2298.7378  1156.0399  1935.8038  1136.4219
 2234.979   1106.9948  1876.9497   891.19617 2102.5574   871.5781
 1847.5226   641.0659  1999.5625   660.684   1636.6284  1048.1406
 1798.4775  1013.8091  1415.9253  1048.1406  1729.8142   979.4774
 1415.9253  1371.8385  1680.769   1293.3662  2146.698   1170.7534
 1886.7587   596.92535 1975.0399   621.4479  1979.9445  1087.3767
 1857.3315  1048.1406  1366.8802  1406.1702  1651.342   1317.8889 ]


100%|██████████| 1600/1600 [02:58<00:00,  8.95it/s]


[3300.9714  1572.2109  3281.3535  1645.7787  3369.6348  1626.1605
 3232.308   1783.1051  3448.1067  1738.9644  2903.7058  1704.6328
 3345.1118  1665.3967  2815.4246  1341.6987  3158.7405  1307.3672
 2771.284    964.05115 3001.796   1008.19183 2457.395   1572.2109
 2697.7163  1523.1659  2123.888   1572.2109  2604.5303  1469.2161
 2123.888   2057.7578  2521.1536  1935.1448  3212.6902  1753.678
 2825.2334   900.29254 2962.5598   939.52856 2967.4644  1635.9696
 2785.9973  1572.2109  2050.3203  2106.803   2477.0132  1979.2856 ]


100%|██████████| 1600/1600 [02:59<00:00,  8.93it/s]


[4406.1997 2096.2812 4376.773  2179.658  4494.481  2164.9443 4303.205
 2366.0295 4597.4756 2307.1753 3871.6077 2272.8438 4455.2446 2209.085
 3753.8994 1787.2969 4210.019  1743.1562 3695.0452 1287.0364 4008.9338
 1326.2726 3273.257  2096.2812 3596.955  2032.5227 2826.946  2096.2812
 3464.533  1958.9548 2831.8506 2743.677  3361.538  2576.9233 4278.6826
 2341.5068 3763.7083 1203.6597 3950.0798 1252.7048 3954.9844 2169.8489
 3714.663  2091.3767 2733.7605 2812.3403 3302.684  2635.7778]


100%|██████████| 1600/1600 [02:59<00:00,  8.92it/s]
<ipython-input-25-1cb1638a5783>:69: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1:] = all_predictions


[5506.5234 2625.256  5472.1924 2728.251  5624.232  2708.6328 5383.9106
 2953.8584 5746.8447 2875.3862 4839.51   2841.0547 5565.3774 2752.7734
 4697.279  2232.895  5261.298  2183.8499 4618.8066 1610.0216 5011.1675
 1673.7804 4089.1191 2620.3516 4491.289  2541.8794 3534.9087 2620.3516
 4319.631  2448.6936 3539.8132 3429.5962 4197.018  3223.6064 5344.675
 2919.5269 4707.0874 1502.1223 4937.5996 1565.8811 4942.5044 2708.6328
 4643.329  2615.447  3417.2007 3517.8777 4133.26   3292.27  ]
[1101.3047   525.0512  1094.4385   545.6502  1124.8464   541.72656
 1076.7821   590.77167 1149.3689   575.0773   967.902    568.21094
 1113.0754   550.5547   939.45575  446.579   1052.2595   436.76996
  923.76135  322.00433 1002.2335   334.75607  817.82385  524.0703
  898.2578   508.3759   706.98175  524.0703   863.92615  489.7387
  707.96265  685.91925  839.4036   644.7213  1068.9349   583.9054
  941.4175   300.42447  987.5199   313.1762   988.50085  541.72656
  928.66583  523.0894   683.4401   703.57556  8